<a href="https://colab.research.google.com/github/hzhe5009/CS36-Capstone-LSTM/blob/master/LSTM_timestep_60(Huixian_Zheng_480070588).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LeakyReLU,Dropout, LSTM, concatenate,Input
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from datetime import datetime, timedelta, time
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = '/content/gdrive/My Drive/ctx/combine_data/partzone3month.csv'
new_select_data = pd.read_csv(data_path)

new_select_data['pickup_time_g_index'] = pd.to_datetime(new_select_data['pickup_time_g']) #, format='%Y/%m/%d %I:%M:%S %p', errors='ignore')
new_select_data['pickup_time_g'] = new_select_data['pickup_time_g_index'].copy()
new_select_data.set_index('pickup_time_g_index',inplace=True)
new_select_grouper = new_select_data.groupby('LocationID')
train_data, label_data = [],[]
for each_loc in new_select_grouper:
  location_data = each_loc[1]
  first_time = True
  location_data_sample = location_data.sample(frac=0.001, random_state=2020)
  for index, row in location_data_sample.iterrows():
    this_time = row['pickup_time_g']
    time_window = (location_data['pickup_time_g']>=this_time - timedelta(minutes=60))&(location_data['pickup_time_g']<=this_time - timedelta(minutes=1))
    window_items = location_data[time_window]
    if len(window_items) == 60:
        pu_day_cat = to_categorical(window_items['day'], num_classes=32)
        pu_week_cat = to_categorical(window_items['weekday'], num_classes=7)
        hour_cat = to_categorical(window_items['hour'], num_classes=25)
        minute_cat = to_categorical(window_items['minute'], num_classes=60)
        if first_time:
          first_time = False
          locat_cat = to_categorical(window_items['LocationID'], num_classes=265)
        train_data.append(np.hstack((pu_day_cat,pu_week_cat,hour_cat,minute_cat,locat_cat,
                                    window_items['minute_demand'].values.reshape(-1,1))))
        label_data.append(row['15_demand'])

train_data = np.array(train_data)
label_data = np.array(label_data)
X_tr, X_val, y_tr, y_val = train_test_split(train_data, label_data, test_size=0.1, random_state=2020)

In [0]:
# define model
model = Sequential()
model.add(LSTM(512, batch_input_shape=(None, 60, 7+32+25+60+1+265)))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))
model.add(Dense(265))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu'))
print(model.summary())

# compile the model
model.compile(loss='mean_squared_error', metrics=['mse', 'mae', 'mape'], optimizer='adam')
history = model.fit(X_tr, y_tr, epochs=10, verbose=2, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               1849344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 265)               1

KeyboardInterrupt: ignored

In [0]:
# evaluate the model
train_metric = model.evaluate(X_tr, y_tr, verbose=2)
test_metric = model.evaluate(X_val, y_val, verbose=2)
print(train_metric)
print(test_metric)

472/472 - 63s - loss: 24.1581 - mse: 24.1721 - mae: 2.3187 - mape: 28256920.0000
53/53 - 7s - loss: 27.8973 - mse: 27.6960 - mae: 2.6127 - mape: 38469472.0000
[24.158143997192383, 24.172054290771484, 2.318727493286133, 28256920.0]
[27.897294998168945, 27.696016311645508, 2.6126868724823, 38469472.0]


In [0]:
# plot loss during training
pyplot.title('Loss / Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [0]:
data_path = '/content/gdrive/My Drive/ctx/combine_data/2019test/All_Demand_2019_3months.csv'
new_select_data = pd.read_csv(data_path)

new_select_data['pickup_time_g_index'] = pd.to_datetime(new_select_data['pickup_time_g']) #, format='%Y/%m/%d %I:%M:%S %p', errors='ignore')
new_select_data['pickup_time_g'] = new_select_data['pickup_time_g_index'].copy()
new_select_data.set_index('pickup_time_g_index',inplace=True)
new_select_grouper = new_select_data.groupby('LocationID')
train_data, label_data = [],[]
for each_loc in new_select_grouper:
  location_data = each_loc[1]
  first_time = True
  location_data_sample = location_data.sample(frac=0.001, random_state=2020)
  for index, row in location_data_sample.iterrows():
    this_time = row['pickup_time_g']
    time_window = (location_data['pickup_time_g']>=this_time - timedelta(minutes=60))&(location_data['pickup_time_g']<=this_time - timedelta(minutes=1))
    window_items = location_data[time_window]
    if len(window_items) == 60:
        pu_day_cat = to_categorical(window_items['day'], num_classes=32)
        pu_week_cat = to_categorical(window_items['weekday'], num_classes=7)
        hour_cat = to_categorical(window_items['hour'], num_classes=25)
        minute_cat = to_categorical(window_items['minute'], num_classes=60)
        if first_time:
          first_time = False
          locat_cat = to_categorical(window_items['LocationID'], num_classes=265)
        train_data.append(np.hstack((pu_day_cat,pu_week_cat,hour_cat,minute_cat,locat_cat,
                                    window_items['minute_demand'].values.reshape(-1,1))))
        label_data.append(row['15_demand'])


train_data = np.array(train_data)
label_data = np.array(label_data)
X_tr, X_val, y_tr, y_val = train_test_split(train_data, label_data,  test_size=0.1, random_state=2020)

In [0]:
X_tr, X_val, y_tr, y_val = train_test_split(train_data, label_data,  test_size=0.1, random_state=2020)

In [0]:
test_metric = model.evaluate(X_val, y_val, verbose=2)
print(test_metric)

53/53 - 7s - loss: 27.0087 - mse: 27.2514 - mae: 2.4817 - mape: 46436644.0000
[27.008668899536133, 27.251371383666992, 2.4816675186157227, 46436644.0]


In [0]:
pyplot.plot(history.history['mean_squared_error'])
# pyplot.plot(history.history['mean_absolute_error'])
# pyplot.plot(history.history['mean_absolute_percentage_error'])
pyplot.show()

In [0]:
# pyplot.plot(history.history['mean_squared_error'])
pyplot.plot(history.history['mean_absolute_error'])
# pyplot.plot(history.history['mean_absolute_percentage_error'])
pyplot.show()